In [1]:
import os
from dotenv import load_dotenv

load_dotenv()
VANTAGE_KEY = os.environ['VANTAGE_API']
assert VANTAGE_KEY is not None

In [3]:
import polars as pl
sp500_table = pl.read_csv('data/sp500.csv').with_row_index('company_id')
test_symbols = list(sp500_table.sample(5)['Symbol'])

In [4]:
from airflow.models import Variable
# Function to get the next batch of companies
def get_next_batch(batch_size=25):
    company_list = sp500_table

    # Load the last processed index from Airflow Variable (or other storage like a file)
    last_processed_index = Variable.get('last_processed_index', default_var=0)
    last_processed_index = int(last_processed_index)

    # Get the next batch of companies
    next_batch = company_list[last_processed_index:last_processed_index + batch_size, "Symbol"]

    # Update the last processed index and store it back in Airflow Variable
    new_last_index = last_processed_index + len(next_batch)
    Variable.set('last_processed_index', new_last_index)

    return next_batch

In [22]:
import boto3
import json

In [24]:
s3_client = boto3.client('s3')
s3_client.put_object(
    Bucket='mote-finder',
    Key='raw/test_json.json',
    Body=json.dumps(balance_data),
    ContentType='application/json'
)

{'ResponseMetadata': {'RequestId': 'N49BZZ7KG39T7X7X',
  'HostId': '+mDclJckdhQ5ZBuUVTzG+daJ70vf3YFv8KdIaOhxyIbu0UqyKxCIV01vFXrB2uCMnkeEY0edhex9EFA+j5NKyKOcMTENFKDccoUO8IZdZ2c=',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amz-id-2': '+mDclJckdhQ5ZBuUVTzG+daJ70vf3YFv8KdIaOhxyIbu0UqyKxCIV01vFXrB2uCMnkeEY0edhex9EFA+j5NKyKOcMTENFKDccoUO8IZdZ2c=',
   'x-amz-request-id': 'N49BZZ7KG39T7X7X',
   'date': 'Fri, 27 Sep 2024 01:06:17 GMT',
   'x-amz-server-side-encryption': 'AES256',
   'etag': '"f3d36bab39f0776eec34b6d104c3bf6c"',
   'server': 'AmazonS3',
   'content-length': '0'},
  'RetryAttempts': 0},
 'ETag': '"f3d36bab39f0776eec34b6d104c3bf6c"',
 'ServerSideEncryption': 'AES256'}

In [13]:
# import requests
# income_data = []
# for sym in test_symbols:
#     url = f'https://www.alphavantage.co/query?function=INCOME_STATEMENT&symbol={sym}&apikey={VANTAGE_KEY}'
#     r = requests.get(url)
#     income_data.append(r.json())

balance_data = []
for sym in test_symbols:
    url = f'https://www.alphavantage.co/query?function=BALANCE_SHEET&symbol={sym}&apikey={VANTAGE_KEY}'
    r = requests.get(url)
    balance_data.append(r.json())



In [18]:
with open('data/sp500.csv', 'r') as f:
    companies = [line.strip() for line in f.readlines()]

In [19]:
companies

['Symbol,Name,Sector',
 'MMM,3M Company,Industrials',
 'AOS,A.O. Smith Corp,Industrials',
 'ABT,Abbott Laboratories,Health Care',
 'ABBV,AbbVie Inc.,Health Care',
 'ACN,Accenture plc,Information Technology',
 'ATVI,Activision Blizzard,Information Technology',
 'AYI,Acuity Brands Inc,Industrials',
 'ADBE,Adobe Systems Inc,Information Technology',
 'AAP,Advance Auto Parts,Consumer Discretionary',
 'AMD,Advanced Micro Devices Inc,Information Technology',
 'AES,AES Corp,Utilities',
 'AET,Aetna Inc,Health Care',
 'AMG,Affiliated Managers Group Inc,Financials',
 'AFL,AFLAC Inc,Financials',
 'A,Agilent Technologies Inc,Health Care',
 'APD,Air Products & Chemicals Inc,Materials',
 'AKAM,Akamai Technologies Inc,Information Technology',
 'ALK,Alaska Air Group Inc,Industrials',
 'ALB,Albemarle Corp,Materials',
 'ARE,Alexandria Real Estate Equities Inc,Real Estate',
 'ALXN,Alexion Pharmaceuticals,Health Care',
 'ALGN,Align Technology,Health Care',
 'ALLE,Allegion,Industrials',
 'AGN,"Allergan, Plc

In [14]:
balance_list = []

# Loop through each company's API response
for response in balance_data:
    # Extract annualReports data
    df = pl.DataFrame(response['annualReports'])
    
    # Add the company symbol as a new column
    df = df.with_columns(
        pl.lit(response['symbol']).alias("company_id"),   
    )
    
    # Append the DataFrame to the list
    balance_list.append(df)

# Combine all DataFrames into one
balance_df = pl.concat(balance_list)

In [10]:
income_list = []

# Loop through each company's API response
for response in income_data:
    # Extract annualReports data
    df = pl.DataFrame(response['annualReports'])
    
    # Add the company symbol as a new column
    df = df.with_columns(
        pl.lit(response['symbol']).alias("company_id"),   
    )
    
    # Append the DataFrame to the list
    income_list.append(df)

# Combine all DataFrames into one
income_df = pl.concat(income_list)


In [15]:
balance_df

fiscalDateEnding,reportedCurrency,totalAssets,totalCurrentAssets,cashAndCashEquivalentsAtCarryingValue,cashAndShortTermInvestments,inventory,currentNetReceivables,totalNonCurrentAssets,propertyPlantEquipment,accumulatedDepreciationAmortizationPPE,intangibleAssets,intangibleAssetsExcludingGoodwill,goodwill,investments,longTermInvestments,shortTermInvestments,otherCurrentAssets,otherNonCurrentAssets,totalLiabilities,totalCurrentLiabilities,currentAccountsPayable,deferredRevenue,currentDebt,shortTermDebt,totalNonCurrentLiabilities,capitalLeaseObligations,longTermDebt,currentLongTermDebt,longTermDebtNoncurrent,shortLongTermDebtTotal,otherCurrentLiabilities,otherNonCurrentLiabilities,totalShareholderEquity,treasuryStock,retainedEarnings,commonStock,commonStockSharesOutstanding,company_id
str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str
"""2023-12-31""","""USD""","""9686522000""","""5588996000""","""2297675000""","""3253280000""","""971406000""","""1193964000""","""4167518000""","""890796000""","""261235000""","""2845080000""","""1427139000""","""1417941000""","""1032036000""","""76431000""","""955605000""","""170346000""","""110216000""","""1457778000""","""1161689000""","""564379000""","""492365000""","""6449000""","""17537000""","""344567000""","""19000""","""None""","""None""","""None""","""66015000""","""537859000""","""91838000""","""8228744000""","""2566383000""","""5939736000""","""5613000""","""1041571000""","""MNST"""
"""2022-12-31""","""USD""","""8293105000""","""4764897000""","""1307141000""","""2669455000""","""935631000""","""1016203000""","""3567818000""","""516897000""","""233158000""","""2638351000""","""1220410000""","""1417941000""","""1423757000""","""61443000""","""1362314000""","""109823000""","""134478000""","""1268064000""","""1001978000""","""444265000""","""534211000""","""757000""","""16303999""","""266086000""","""41000""","""None""","""None""","""None""","""45931000""","""498098001""","""42286000""","""7025041000""","""6600281000""","""9001173000""","""3209000""","""522300000""","""MNST"""
"""2021-12-31""","""USD""","""7804784000""","""4682110000""","""1326462000""","""3076189000""","""593357000""","""896658000""","""3148477000""","""313753000""","""224785000""","""2404029000""","""1072386000""","""1331643000""","""1849146000""","""99419000""","""1749727000""","""115906000""","""80252000""","""1237833000""","""965076000""","""404263000""","""571579000""","""None""","""10750000""","""272757000""","""None""","""None""","""None""","""None""","""28180000""","""507533000""","""29508000""","""6566951000""","""5829253000""","""7809549000""","""3200000""","""529323000""","""MNST"""
"""2020-12-31""","""USD""","""6202716000""","""3140955000""","""1180413000""","""2061767000""","""333085000""","""666012000""","""3086635000""","""314656000""","""204754000""","""2390689000""","""1059046000""","""1331643000""","""925645000""","""44291000""","""881354000""","""80091000""","""70475000""","""1041856000""","""749988000""","""296800000""","""619765000""","""None""","""2402000""","""291868000""","""None""","""None""","""None""","""None""","""18141000""","""405357000""","""27432000""","""5160860000""","""5815423000""","""6432074000""","""3193000""","""528097000""","""MNST"""
"""2019-12-31""","""USD""","""5150352000""","""2316309000""","""797957000""","""1331020000""","""360731000""","""540330000""","""2868017000""","""298640000""","""207434000""","""2383748000""","""1052105000""","""1331643000""","""None""","""12905000""","""533063000""","""84228000""","""53973000""","""979071000""","""661097000""","""274045000""","""663406000""","""4297000""","""3661000""","""343625000""","""None""","""None""","""2812000""","""None""","""28463000""","""339154000""","""30505000""","""4171281000""","""5219505000""","""5022480000""","""3182000""","""536698000""","""MNST"""
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,

In [12]:
income_df

fiscalDateEnding,reportedCurrency,grossProfit,totalRevenue,costOfRevenue,costofGoodsAndServicesSold,operatingIncome,sellingGeneralAndAdministrative,researchAndDevelopment,operatingExpenses,investmentIncomeNet,netInterestIncome,interestIncome,interestExpense,nonInterestIncome,otherNonOperatingIncome,depreciation,depreciationAndAmortization,incomeBeforeTax,incomeTaxExpense,interestAndDebtExpense,netIncomeFromContinuingOperations,comprehensiveIncomeNetOfTax,ebit,ebitda,netIncome,company_id
str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str
"""2023-12-31""","""USD""","""3794206000""","""7079406000""","""4056680000""","""3345821000""","""1953355000""","""597700000""","""None""","""1840851000""","""None""","""-255000""","""None""","""255000""","""7079661000""","""115127000""","""None""","""70159000""","""2068482000""","""437494000""","""255000""","""1630988000""","""1664724000""","""1953355000""","""2138896000""","""1630988000""","""MNST"""
"""2022-12-31""","""USD""","""3174567000""","""6273311000""","""3725328000""","""3136483000""","""1584721000""","""524800000""","""None""","""1589846000""","""None""","""-24000""","""None""","""24000""","""6273335000""","""-12757000""","""None""","""61845000""","""1571964000""","""380340000""","""24000""","""1191624000""","""1101716000""","""1584721000""","""1633833000""","""1191624000""","""MNST"""
"""2021-12-31""","""USD""","""3108513000""","""5542046000""","""2971585000""","""2432839000""","""1797467000""","""488100000""","""None""","""1311046000""","""None""","""-19000""","""None""","""19000""","""5542065000""","""3952000""","""None""","""50646000""","""1801419000""","""423944000""","""19000""","""1377475000""","""1305276000""","""1797467000""","""1852084000""","""1377475000""","""MNST"""
"""2020-12-31""","""USD""","""2723880000""","""4587749000""","""2357084000""","""1874758000""","""1633153000""","""416000000""","""None""","""1090727000""","""None""","""-39000""","""None""","""39000""","""4587788000""","""-6996000""","""None""","""57626000""","""1626157000""","""216563000""","""39000""","""1409594000""","""1445015000""","""1633153000""","""1683822000""","""1409594000""","""MNST"""
"""2019-12-31""","""USD""","""2518585000""","""4196915000""","""2198370000""","""1682234000""","""1402939000""","""455000000""","""None""","""1115646000""","""None""","""-56000""","""13023000""","""56000""","""4196971000""","""13023000""","""None""","""61136000""","""1415962000""","""308127000""","""56000""","""1107835000""","""1108312000""","""1402939000""","""1477154000""","""1107835000""","""MNST"""
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
"""2013-12-31""","""USD""","""10691000000""","""38856000000""","""34810000000""","""28364000000""","""2636000000""","""5190000000""","""1804000000""","""8220000000""","""69000000""","""-327000000""","""None""","""327000000""","""128000000""","""238000000""","""670000000""","""319000000""","""5374000000""","""1450000000""","""327000000""","""3924000000""","""6083000000""","""5701000000""","""6020000000""","""3924000000""","""HON"""
"""2012-12-31""","""USD""","""9374000000""","""37278000000""","""34454000000""","""28291000000""","""299000000""","""5218000000""","""1847000000""","""9426000000""","""58000000""","""-351000000""","""None""","""351000000""","""-36000000""","""70000000""","""660000000""","""266000000""","""3870000000""","""944000000""","""351000000""","""2926000000""","""3031000000""","""4221000000""","""4487000000""","""2926000000""","""HON"""
"""2011-12-31""","""USD""","""7973000000""","""36529000000""","""28556000000""","""28556000000""","""-2042000000""","""5399000000""","""1799000000""","""10015000000""","""58000000""","""-376000000""","""None""","""376000000""","""-50000000""","""84000000""","""699000000""","""249000000""","""2484000000""","""417000000""","""376000000""","""1858000000""","""1694000000""","""2860000000""","""3109000000""","""2067000000""","""HON"""


In [12]:
final_df = final_df.with_columns(
    pl.col('fiscalDateEnding').str.to_date('%Y-%m-%d'),
    pl.col("*").exclude('fiscalDateEnding', 'reportedCurrency', 'company_id').cast(pl.Float64, strict=False)
)

In [20]:
final_df.with_columns(
    (pl.col('incomeTaxExpense')/pl.col('incomeBeforeTax')).alias('taxRate')
    
).with_columns(
    (pl.col('operatingIncome')*(1-pl.col('taxRate'))).alias('NOPAT')
)

fiscalDateEnding,reportedCurrency,grossProfit,totalRevenue,costOfRevenue,costofGoodsAndServicesSold,operatingIncome,sellingGeneralAndAdministrative,researchAndDevelopment,operatingExpenses,investmentIncomeNet,netInterestIncome,interestIncome,interestExpense,nonInterestIncome,otherNonOperatingIncome,depreciation,depreciationAndAmortization,incomeBeforeTax,incomeTaxExpense,interestAndDebtExpense,netIncomeFromContinuingOperations,comprehensiveIncomeNetOfTax,ebit,ebitda,netIncome,company_id,taxRate,NOPAT
date,str,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,str,f64,f64
2023-12-31,"""USD""",7.3790e9,1.1434e10,4.0550e9,1.1700e9,2.8510e9,6.6e7,null,8.49e8,null,-7.2200e8,null,7.22e8,1.2156e10,1.91e8,1.2980e9,null,2.3200e9,4.93e8,7.22e8,1.8270e9,1.8580e9,3.0420e9,null,1.8270e9,"""NSC""",0.2125,2.2452e9
2022-12-31,"""USD""",7.9200e9,1.2053e10,4.1330e9,1.4590e9,4.8090e9,5.3e7,null,6.84e8,null,-6.9200e8,null,6.92e8,1.2745e10,1.3e7,1.2210e9,null,4.1300e9,8.6e8,6.92e8,3.2700e9,3.3210e9,4.8220e9,null,3.2700e9,"""NSC""",0.208232,3.8076e9
2021-12-31,"""USD""",7.2010e9,1.0496e10,3.2950e9,7.99e8,4.4470e9,5.4e7,null,5.15e8,null,-6.4600e8,null,6.46e8,1.1142e10,7.7e7,1.1810e9,null,3.8780e9,8.73e8,6.46e8,3.0050e9,3.1970e9,4.5240e9,null,3.0050e9,"""NSC""",0.225116,3.4459e9
2020-12-31,"""USD""",6.1290e9,9.1640e9,3.0350e9,5.35e8,3.0020e9,2.8e7,null,7.41e8,null,-6.2500e8,null,6.25e8,9.7890e9,1.53e8,1.1540e9,null,2.5300e9,5.17e8,6.25e8,2.0130e9,1.9100e9,3.1550e9,null,2.0130e9,"""NSC""",0.204348,2.3885e9
2019-12-31,"""USD""",6.8860e9,1.0692e10,3.8060e9,9.53e8,3.9890e9,5.3e7,null,8e8,null,-6.0400e8,null,6.04e8,1.1296e10,1.06e8,1.1380e9,null,3.4910e9,7.69e8,6.04e8,2.7220e9,2.7940e9,4.0950e9,null,2.7220e9,"""NSC""",0.220281,3.1103e9
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
2013-12-31,"""USD""",3.5970e9,1.4968e10,1.1306e10,3.08e8,-5.8430e9,7.0630e9,null,9.7830e9,null,3.0090e9,3.3520e9,3.43e8,1.1959e10,1.83e8,null,3.42e8,3.6960e9,1.5920e9,3.43e8,2.1040e9,1.8550e9,4.0390e9,4.3810e9,2.1040e9,"""BK""",0.430736,-3.3262e9
2012-12-31,"""USD""",3.1460e9,1.4534e10,1.1333e10,4.54e8,-6.6820e9,7.0140e9,null,1.0362e10,null,2.9730e9,3.5070e9,5.34e8,1.1561e10,1.89e8,null,3.84e8,3.2790e9,8.42e8,5.34e8,2.4370e9,3.4210e9,3.8130e9,4.1970e9,2.4370e9,"""BK""",0.256786,-4.9662e9
2011-12-31,"""USD""",3.5680e9,1.4680e10,1.1112e10,6.05e8,-6.0410e9,6.9950e9,null,1.0213e10,null,2.9840e9,3.5880e9,6.04e8,1.1696e10,2e8,null,4.28e8,3.5640e9,1.0480e9,6.04e8,2.5160e9,2.2440e9,4.1680e9,4.5960e9,2.5160e9,"""BK""",0.294052,-4.2646e9
